In [1]:
# Q11. MTK ver
# 모듈 불러오기
using ModelingToolkit
using DifferentialEquations
using Sundials
using Plots
using ModelingToolkit: t_nounits as t, D_nounits as D, @variables, @parameters, @named, @unpack, @mtkbuild
using Symbolics
using BenchmarkTools
using Profile, ProfileView

# HX_PHX 자료구조
struct Wall
    rho::Float64
    V::Float64
    m::Float64
    Cp::Float64
    m_elem::Float64
end
struct Ref
    V::Float64
    A_cs::Float64
    V_elem::Float64
    A_elem::Float64
    Ge1::Float64
    Ge2::Float64
end
struct Fld
    V::Float64
    A_cs::Float64
    V_elem::Float64
    A_elem::Float64
    k::Float64
    Pr::Float64
    rho::Float64
    mu::Float64
    Cp::Float64
    mdot::Float64
    T_in::Float64
end
struct HX_PHX
    N_t::Int64
    N_c::Int64
    N_f::Int64
    L_w::Float64
    L_v::Float64
    A_t::Float64
    phi::Float64
    p::Float64
    t::Float64
    b::Float64
    D_h::Float64
    beta::Float64
    N_CV::Int64
    Wall::Wall
    Ref::Ref
    Fld::Fld
    isCounterFlow::Bool
end
# make_HX ftn
function make_HX(N_t::Int64, L_w::Float64, L_v::Float64, A_t::Float64, p::Float64, t::Float64, N_CV::Int64, isCounterFlow::Bool)
    # DIC-193-33 PHX. http://www.dibr.co.kr/exchanger/detail/D93RP00056?ts=1735978818&capacity=0.5&cate=162&
    N_t = N_t
    N_c = (N_t - 1) / 2
    N_f = (N_t - 1) / 2
    L_w = L_w
    L_v = L_v
    A_t = A_t
    ϕ = A_t / (L_w * L_v * (N_t - 2))
    p = p
    t = t
    b = p - t
    D_h = 2 * b / ϕ
    β = 25 * pi / 180
    N_CV = N_CV

    # Wall Geo, Props
    Wall_rho = 7.93e3 # STS 304 density
    Wall_V = A_t * t
    Wall_m = Wall_V * Wall_rho
    Wall_Cp = 502 # specific heat

    # Ref, Fld Geo
    Ref_V = N_c * L_w * b* L_v
    Ref_A_cs = N_c * L_w * b
    Fld_V =  N_f * L_w * b * L_v
    Fld_A_cs = N_f * L_w * b

    # Geo per Element
    Wall_m_elem = Wall_m / N_CV
    Ref_V_elem = Ref_V / N_CV
    Ref_A_elem = A_t / N_CV
    Fld_V_elem = Fld_V / N_CV
    Fld_A_elem = Ref_A_elem

    # Ref UA Props
    Ref_Ge1 = 11.22*(b/D_h)^(-2.83) * (pi/2 - β)^(-4.5)
    Ref_Ge2 = 0.35*(b/D_h)^(0.23) * (pi/2 - β)^(1.48)

    # Fld Props (Water, 15C, Saturated Liquid, refprop)
    Fld_k = 0.5887
    Fld_Pr = 1.7058
    Fld_rho = 999.0561
    Fld_mu = 0.0011
    Fld_Cp = 4.1888e3

    # Fld
    Fld_mdot = 0.1
    Fld_T_in = 273.15 + 40
    isCounterFlow = isCounterFlow

    HX_Wall = Wall(Wall_rho, Wall_V, Wall_m, Wall_Cp, Wall_m_elem)
    HX_Ref = Ref(Ref_V, Ref_A_cs, Ref_V_elem, Ref_A_elem, Ref_Ge1, Ref_Ge2)
    HX_Fld = Fld(Fld_V, Fld_A_cs, Fld_V_elem, Fld_A_elem, Fld_k, Fld_Pr, Fld_rho, Fld_mu, Fld_Cp, Fld_mdot, Fld_T_in)
    return HX_PHX(N_t, N_c, N_f, L_w, L_v, A_t, ϕ, p, t, b, D_h, β, N_CV, HX_Wall, HX_Ref, HX_Fld, isCounterFlow)
end

make_HX (generic function with 1 method)

In [2]:
# HX_C
HX_C_N_t = 33
HX_C_L_w = 0.083
HX_C_L_v = 0.154
HX_C_A_t = 0.434
HX_C_p = 2.35e-3
HX_C_t = 0.4e-3
HX_C_N_CV = 20
HX_C_isCounterFlow = true
HX_C = make_HX(HX_C_N_t, HX_C_L_w, HX_C_L_v, HX_C_A_t, HX_C_p, HX_C_t, HX_C_N_CV, HX_C_isCounterFlow)

HX_PHX(33, 16, 16, 0.083, 0.154, 0.434, 1.095290251916758, 0.00235, 0.0004, 0.0019500000000000001, 0.0035607, 0.4363323129985824, 20, Wall(7930.0, 0.00017360000000000002, 1.376648, 502.0, 0.0688324), Ref(0.0003987984000000001, 0.0025896000000000005, 1.9939920000000003e-5, 0.0217, 34.95278503589673, 0.3672934117781399), Fld(0.0003987984000000001, 0.0025896000000000005, 1.9939920000000003e-5, 0.0217, 0.5887, 1.7058, 999.0561, 0.0011, 4188.8, 0.1, 313.15), true)

In [28]:
# CoolProp 관련 함수
import CoolProp as CP
# CoolProp with MTK (Tabular / Consider to change into PINN)
 # ρ, T from P, h
const handle = CP.AbstractState_factory("HEOS&BICUBIC", "R410A")
const HmassP_inputs = CP.get_input_pair_index("HmassP_INPUTS")
const Dmass_output = CP.get_param_index("Dmass")
const T_output = CP.get_param_index("T")

function R410a_ρ(P, h)
    CP.AbstractState_update(handle, HmassP_inputs, h, P)   
    return CP.AbstractState_keyed_output(handle, Dmass_output)
end
function R410a_T(P, h)
    CP.AbstractState_update(handle, HmassP_inputs, h, P)   
    return CP.AbstractState_keyed_output(handle, T_output)
end

function R410a_L(P)
    CP.AbstractState_update(handle, CP.get_input_pair_index("PQ_INPUTS"), P, 0)
    h_L = CP.AbstractState_keyed_output(handle, CP.get_param_index("Hmass"))
    ρ_L = CP.AbstractState_keyed_output(handle, CP.get_param_index("Dmass"))
    k_L = CP.AbstractState_keyed_output(handle, CP.get_param_index("L"))
    mu_L = CP.AbstractState_keyed_output(handle, CP.get_param_index("V"))
    Pr_L = CP.AbstractState_keyed_output(handle, CP.get_param_index("Prandtl"))
    return (h_L, ρ_L, k_L, mu_L, Pr_L)
end
function R410a_V(P)
    CP.AbstractState_update(handle, CP.get_input_pair_index("PQ_INPUTS"), P, 1)
    h_V = CP.AbstractState_keyed_output(handle, CP.get_param_index("Hmass"))
    ρ_V = CP.AbstractState_keyed_output(handle, CP.get_param_index("Dmass"))
    return (h_V, ρ_V)
end

function R410a_dρdP_h(P,h)
    return (R410a_ρ(P+0.5, h) - R410a_ρ(P-0.5, h))
end
function R410a_dρdh_P(P,h)
    return (R410a_ρ(P, h+0.5) - R410a_ρ(P, h-0.5))
end

# Symbolics
@register_symbolic R410a_ρ(P, h) 
@register_symbolic R410a_T(P, h)

@register_symbolic R410a_dρdP_h(P,h)
@register_symbolic R410a_dρdh_P(P,h)

# Symbolics_Der
Symbolics.derivative(::typeof(R410a_ρ), args::NTuple{2,Any}, ::Val{1}) = R410a_ρ(args[1]+0.5, args[2]) -  R410a_ρ(args[1]-0.5, args[2])
Symbolics.derivative(::typeof(R410a_ρ), args::NTuple{2,Any}, ::Val{2}) = R410a_ρ(args[1], args[2]+0.5) -  R410a_ρ(args[1], args[2]-0.5)

# Helper ftns
# 1. regPow
# (Qiao et al. 2022), regPow_gen
function regPow(x, a, b, δ)
    return x^b * (x*x + δ*δ)^((a-b) / 2)
end
function regSqrt(x)
    return regPow(x, 0.5, 3, 0.001)
end
# 2. Func for UA_elem
function U_func(P, h, mdot_in, HX_Arr)
    (h_L, ρ_L, k_L, mu_L, Pr_L) = R410a_L(P)
    (h_V, ρ_V) = R410a_V(P)
    x = (h - h_L) / (h_V - h_L)

    HX_D_h, HX_beta, HX_Ref_A_cs, HX_Ref_Ge1, HX_Ref_Ge2 = @view HX_Arr[1:5]

    # Single phase 
    G = mdot_in / HX_Ref_A_cs
    Re_L = G * HX_D_h / mu_L
    U_ref_single = 0.295 * (k_L/HX_D_h) * ifelse(Re_L >= 0.0, regPow(Re_L, 0.64, 3, 0.1), 0.0) * Pr_L^0.32 * (π/2 - HX_beta)^0.09

    # Two phase
    G_eq_tp = G * (1-x + x*(ρ_L/ρ_V)^0.5)
    Re_eq_tp = G_eq_tp * HX_D_h / mu_L
    U_ref_tp = HX_Ref_Ge1 * (k_L/HX_D_h) * ifelse(Re_eq_tp >= 0.0, regPow(Re_eq_tp, HX_Ref_Ge2, 3, 0.1), 0.0) * Pr_L^(1/3)

    slope1 = (U_ref_tp - U_ref_single) / 0.1
    slope2 = (U_ref_single - U_ref_tp) / 0.1 # slope2 == -slope1

    # Smoothing with ifelse
    U_ref = U_ref_single +
           ifelse(x > 0.0, slope1 * (x - 0.0), 0) +      # x=0, 0 -> slope1
           ifelse(x > 0.1, -slope1 * (x - 0.1), 0) +     # x=0.1, slope1 -> 0
           ifelse(x > 0.9, slope2 * (x - 0.9), 0) +      # x=0.9, 0 -> slope2
           ifelse(x > 1.0, -slope2 * (x - 1.0), 0)       # x=1.0, slope2 -> 0

    return U_ref
end
# 3. Fld UA_elem calc / Const
function U_func_Fld(HX::HX_PHX)
    G = HX.Fld.mdot / HX.Fld.A_cs
    Re_Fld = G * HX.D_h / HX.Fld.mu
    U_ref_Fld = 0.295 * (HX.Fld.k/HX.D_h) * Re_Fld^0.64 * HX.Fld.Pr^0.32 * (π/2 - HX.beta)^0.09
    return U_ref_Fld
end
# 4. Func for h_flow
# Qiao et al. (2015) : Transient modeling of a flash tank vapor injection heat pump system - Part I: Model development
function h_flow_func(P, h)
    (h_L, ρ_L, _, _, _) = R410a_L(P)
    (h_V, ρ_V) = R410a_V(P)
    x = (h - h_L) / (h_V - h_L)
    # Single phase
    if abs(x - 0.5) >= 0.5   
        return h
    end
    ρ = R410a_ρ(P, h)

    # Two phase
    γ = (ρ - ρ_L) / (ρ_V - ρ_L) 
    # Zivi (1964) : c = 1, q = 1, r = (2/3), s = 0
    # c, q, r = 1.0, 1.0, 2/3
    # Smith (1969) : c = 0.79, q = 0.78, r = 0.58, s = 0
    c, q, r = 0.79, 0.78, 0.58
    x_flow = γ^(1/q) / (γ^(1/q) + ((1/c)*(ρ_L / ρ_V)^(r))^(1/q) * (1 - γ)^(1/q) )

    h_flow = h + (x_flow - x) * (h_V - h_L)
    return h_flow
end

# Symbolics
@register_symbolic regPow(x, a, b, δ)
@register_symbolic regSqrt(x)
@register_symbolic U_func(P, h, mdot_in, HX_Arr::Symbolics.Arr{Num,5})
@register_symbolic h_flow_func(P, h)

# Utils
function unpack_sys(sys, sol)
    @unpack P, h, ρ, UA_elem_hat, UA_elem, mdot = sys

    # 1. Vectors for P, h, mdot_out
    all_h_sols = sol[h]
    all_ρ_sols = sol[ρ]
    all_mdot_sols = sol[mdot]
    all_UA_elem_hat_sols = sol[UA_elem_hat]
    all_UA_elem_sols = sol[UA_elem]
    # [t, Vector(P), Vector(h1), Vector(h2), ...] Vector{Vector}
    return vcat([sol.t], [sol[P]], all_h_sols, all_mdot_sols, all_ρ_sols, all_UA_elem_hat_sols, all_UA_elem_sols)
    # return vcat([sol.t], [sol[P]], all_h_sols, all_mdot_sols, all_ρ_sols, all_UA_elem_hat_sols)
end

unpack_sys (generic function with 2 methods)

In [4]:
# 1. Params, Variables for Ref
N = HX_C.N_CV
U_Fld_elem = U_func_Fld(HX_C)

# Init Props
P_in_val = 3200e3
P_out_val = 3000e3
h_in_val = 450e3
c1_val = 9e-6
c2_val = c1_val
Δt_val = 0.01  #[s]

0.01

In [6]:
# 1. Params for HX
# 1-0. Init cond
@parameters begin
    c1=c1_val 
    c2=c2_val
    P_in=P_in_val
    P_out=P_out_val
    h_init=h_in_val 
    ρ_init=R410a_ρ(P_in_val, h_in_val)
    Δt = Δt_val  #[s]
end

# 1-1. Ref side
@parameters begin
    HX_Arr[1:5] = [HX_C.D_h, HX_C.beta, HX_C.Ref.A_cs, HX_C.Ref.Ge1, HX_C.Ref.Ge2]
    HX_Ref_A_elem = HX_C.Ref.A_elem
    HX_Ref_V_elem = HX_C.Ref.V_elem
end
# 1-2. Wall Side
@parameters begin
    HX_Wall_m_elem = HX_C.Wall.m_elem
    HX_Wall_Cp = HX_C.Wall.Cp
end
# 1-3. Fld Side
@parameters begin
    HX_Fld_mdot_in = HX_C.Fld.mdot
    HX_Fld_T_in = HX_C.Fld.T_in
    HX_Fld_A_elem = HX_C.Fld.A_elem
    HX_Fld_Cp = HX_C.Fld.Cp
    UA_Fld_elem = U_Fld_elem * HX_C.Fld.A_elem # UA const for Fld side
end

# 2. Variables for HX
# 2-1. Ref Side
    # Static
@variables begin
    P(t)
    h(t)[1:N]
    ρ(t)[1:N]
    T(t)[1:N]
    UA_elem(t)[1:N]
    UA_elem_hat(t)[1:N]
end
    # Flow
@variables begin
    mdot(t)[1:N+1]
    # mdot_in(t)[1:N]
    # mdot_out(t)[1:N]
    # h_in(t)[1:N]
    # h_out(t)[1:N]
    # H_in(t)[1:N]
    # H_out(t)[1:N]
    Q_r(t)[1:N]
end
# 2-2. Wall Side
@variables T_wall(t)[1:N]
# 2-3. Fld Side
    # Static
@variables begin
    T_Fld(t)[1:N+1]
end
    # Flow
@variables begin
    Q_f(t)[1:N]
end


# 3. Equations
# 3-1. Cond
# 3-1.1 Cond_Ref_side
# Const P method (no friction)
cond_Ref = [
    [
    # Algebraic eqns for Ref
    ρ[i] ~ R410a_ρ(P, h[i]),
    T[i] ~ R410a_T(P, h[i]),
    
    # h_in[i] ~ (i == 1 ? h_flow_func(P_in, h_init) : h_out[i-1]),
    # h_out[i] ~ h_flow_func(P, h[i]),
    # H_in[i] ~ mdot_in[i] * h_in[i],
    # H_out[i] ~ mdot_out[i] * h_out[i],
    
    # Flow vars cal(Q_r[i] : By Static vars (P, h[i], ρ[N]), T)
    # (Qiao et al., 2018) On Closure Relations for Dynamic Vapor Compression Cycle models
    # Low Pass Filtering of UA_elem
    UA_elem_hat[i] ~ U_func(P, h[i], mdot[i], HX_Arr) * HX_Ref_A_elem,
    D(UA_elem[i]) ~ (UA_elem_hat[i] - UA_elem[i]) / Δt,
    Q_r[i] ~ UA_elem[i] * (T_wall[i] - T[i]),
    
    # Differential eqns for Ref [Static vars diff by flow vars(H_in, H_out, Q_r)]
    # (1) Mass conservation
    # HX_Ref_V_elem * D(ρ[i]) ~ mdot_in[i] - mdot_out[i],
    # (2) Energy conservation
    # HX_Ref_V_elem * (D(ρ[i] * h[i]) - D(P[i])) ~ H_in[i] - H_out[i] + Q_r[i]
    # (3) Partial eqns
    # D(ρ[i]) ~ dρdP_h(P, h[i]) * D(P) + dρdh_P(P, h[i]) * D(h[i])
    
    HX_Ref_V_elem * (R410a_dρdP_h(P, h[i]) * D(P) + R410a_dρdh_P(P, h[i]) * D(h[i])) ~ (mdot[i] - mdot[i+1]),
    HX_Ref_V_elem * ((h[i] * R410a_dρdP_h(P, h[i]) - 1) * D(P) + (h[i] * R410a_dρdh_P(P, h[i]) + ρ[i]) * D(h[i])) ~ mdot[i] * (i == 1 ? h_flow_func(P_in, h_init) : h_flow_func(P, h[i-1])) - mdot[i+1] * h_flow_func(P, h[i]) + Q_r[i],
    ]
    for i in 1:N
]
# sqrt, regSqrt options
push!(cond_Ref, [mdot[1] ~ ifelse(P_in - P > 0.0, c1 * (regSqrt(ρ_init * (P_in - P))), 0.0)])
push!(cond_Ref, [mdot[N+1] ~ ifelse(P - P_out > 0.0, c2 * (regSqrt(ρ[N] * (P - P_out))), 0.0)])

# 3-1.2 Cond_Wall_Fld_side
if HX_C.isCounterFlow == true
    #  Cond_Wall_side
    cond_Wall = [
        [
            #  Algebraic eqns for Wall
            #  Differential eqns for Wall
            HX_Wall_m_elem * HX_Wall_Cp * D(T_wall[i]) ~ Q_f[N+1-i] - Q_r[i]
        ]
        for i in 1:N
    ]
    #  Cond_Fld_side
    cond_Fld = [
        [
            #  Algebraic eqns for Fld
            Q_f[N+1-i] ~ (HX_Fld_mdot_in * HX_Fld_Cp) * (1 - exp(-UA_Fld_elem / (HX_Fld_mdot_in * HX_Fld_Cp))) * (T_Fld[N+1-i] - T_wall[i]),
            Q_f[N+1-i] ~ (HX_Fld_mdot_in * HX_Fld_Cp) * (T_Fld[N+1-i] - T_Fld[N+2-i])
        ]
        for i in 1:N
    ]
else
    #  Cond_Wall_side
    cond_Wall = [
        [
            #  Algebraic eqns for Wall
            #  Differential eqns for Wall
            HX_Wall_m_elem * HX_Wall_Cp * D(T_wall[i]) ~ Q_f[i] - Q_r[i]
        ]
        for i in 1:N
    ]
    # Cond_Fld_side
    cond_Fld = [
        [
            #  Algebraic eqns for Fld
            Q_f[i] ~ (HX_Fld_mdot_in * HX_Fld_Cp) * (1 - exp(-UA_Fld_elem / (HX_Fld_mdot_in * HX_Fld_Cp))) * (T_Fld[i] - T_wall[i]),
            Q_f[i] ~ (HX_Fld_mdot_in * HX_Fld_Cp) * (T_Fld[i+1] - T_Fld[i])
        ]
        for i in 1:Nc
    ]
end
push!(cond_Fld, [T_Fld[1] ~ HX_Fld_T_in])


# Flatten the cond models
cond_eqns = vcat(cond_Ref..., cond_Wall..., cond_Fld...)
@mtkcompile sys = ODESystem(cond_model, t; continuous_events = [P - P_out ~ 0])

Model sys:
Equations (82):
  82 standard: see equations(sys)
Unknowns (82): see unknowns(sys)
  (T_wall(t))[20]
  (T_wall(t))[19]
  (T_wall(t))[18]
  (T_wall(t))[17]
  ⋮
Parameters (16): see parameters(sys)
  HX_Wall_Cp [defaults to 502.0]
  HX_Fld_T_in [defaults to 313.15]
  ρ_init [defaults to 119.638]
  HX_Fld_Cp [defaults to 4188.8]
  ⋮
Observed (149): see observed(sys)

In [7]:
mdot_in_val = c1_val * regSqrt(R410a_ρ(P_in_val, h_in_val) * (P_in_val - P_out_val))
HX_Arr_init = [HX_C.D_h, HX_C.beta, HX_C.Ref.A_cs, HX_C.Ref.Ge1, HX_C.Ref.Ge2]
U_init= U_func(P_in_val, h_in_val, mdot_in_val, HX_Arr_init)

# 초기값 정의

u0_P = [P => P_in_val]
u0_h = [h[i] => h_in_val for i in 1:N]
u0_T_wall = [T_wall[i] => HX_C.Fld.T_in for i in 1:N]
u0_UA_elem = [UA_elem[i] => U_init * HX_C.Ref.A_elem for i in 1:N]
u0_map = vcat(u0_P, u0_h, u0_T_wall, u0_UA_elem)
#=
du0_P = [P => 0.0]
du0_h = [h[i] => 0.0 for i in 1:N]
du0_T_wall = [T_wall[i] => 0.0 for i in 1:N]
du0_UA_elem = [UA_elem[i] => 0.0 for i in 1:N]
du0_map = vcat(du0_P, du0_h, du0_T_wall, du0_UA_elem)

u0_P = [P_in_val]
u0_h = [h_in_val for i in 1:N]
u0_T_wall = [HX_C.Fld.T_in for i in 1:N]
u0_UA_elem = [U_init * HX_C.Ref.A_elem for i in 1:N]
u0_map = vcat(u0_P, u0_h, u0_T_wall, u0_UA_elem)

du0_P = [0.0]
du0_h = [0.0 for i in 1:N]
du0_T_wall = [0.0 for i in 1:N]
du0_UA_elem = [0.0 for i in 1:N]
du0_map = vcat(du0_P, du0_h, du0_T_wall, du0_UA_elem)
differential_vars = [true for i in 1:3*N + 1]
=#
# guesses
guess_mdot = [mdot[i] => mdot_in_val for i in 1:N+1]
guess_map = vcat(guess_mdot)

# 문제 정의
tspan = (0.0, 10.0)
prob = ODEProblem(sys, u0_map, tspan, guesses = guess_map)
# prob = DAEProblem(sys, du0_map, u0_map, tspan)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
Initialization status: FULLY_DETERMINED
Non-trivial mass matrix: true
timespan: (0.0, 10.0)
u0: 82-element Vector{Float64}:
 313.15
 313.15
 313.15
 313.15
 313.15
 313.15
 313.15
 313.15
 313.15
 313.15
   ⋮
  11.933680441851802
  11.933680441851802
  11.933680441851802
  11.933680441851802
  11.933680441851802
  11.933680441851802
  11.933680441851802
  11.933680441851802
  11.933680441851802

In [8]:
# For ODEProbelm Rosenbrock23, Rodas5P -> FBDF, QNDF  -> AdaptiveRadau [Stiff ODE solvers]
sol = solve(prob, QNDF(autodiff=false))
# For DAEProbelm IDA() : 실패
# dt 최솟값
# sol = solve(prob, IDA())

retcode: Success
Interpolation: 3rd order Hermite
t: 742-element Vector{Float64}:
  0.0
  9.765625e-10
  1.07421875e-8
  7.669333280862668e-8
  2.7506122668919765e-7
  1.0884813716617052e-6
  6.242761669099401e-6
  1.1397041966537097e-5
  1.4976179414670168e-5
  2.0164146093502193e-5
  ⋮
  8.531669071692175
  8.716763654031805
  8.901858236371435
  9.086952818711065
  9.272047401050695
  9.457141983390326
  9.642236565729956
  9.965278893415837
 10.0
u: 742-element Vector{Vector{Float64}}:
 [313.15, 313.15, 313.15, 313.15, 313.15, 313.15, 313.15, 313.15, 313.15, 313.15  …  11.933680441851802, 11.933680441851802, 11.933680441851802, 11.933680441851802, 11.933680441851802, 11.933680441851802, 11.933680441851802, 11.933680441851802, 11.933680441851802, 11.933680441851802]
 [313.15000000778224, 313.15000000778224, 313.15000000778224, 313.15000000778224, 313.15000000778224, 313.15000000778224, 313.15000000778224, 313.15000000778224, 313.15000000778224, 313.15000000778224  …  11.933680048338

In [ ]:
sol_real=unpack_sys(sys, sol)

3712-element Vector{Vector{Float64}}:
 [0.0, 9.765625e-10, 1.07421875e-8, 7.669333280862668e-8, 2.7506122668919765e-7, 1.0884813716617052e-6, 6.242761669099401e-6, 1.1397041966537097e-5, 1.4976179414670168e-5, 2.0164146093502193e-5  …  8.346574489352545, 8.531669071692175, 8.716763654031805, 8.901858236371435, 9.086952818711065, 9.272047401050695, 9.457141983390326, 9.642236565729956, 9.965278893415837, 10.0]
 [3.2e6, 3.199999995025071e6, 3.1999999375229306e6, 3.1999995411753156e6, 3.199998345970892e6, 3.1999934490111633e6, 3.1999624883854263e6, 3.1999316281990577e6, 3.19991024413499e6, 3.199879302461759e6  …  3.0662760018309345e6, 3.0662776778649553e6, 3.066279204335763e6, 3.066280585196208e6, 3.0662818261046363e6, 3.066282937978237e6, 3.066283930959981e6, 3.066284815602699e6, 3.066286128903538e6, 3.0662862543885577e6]
 [450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000.0, 450000

In [ ]:
function plot_single_run_details(sys, sol, N)
    # 1. 데이터 추출
    @unpack P, h, ρ, UA_elem_hat, UA_elem, mdot = sys

    sol_t = sol.t
    sol_P = sol[P]
    sol_h = sol[h]
    sol_mdot = sol[mdot]
    

    return
end

detailed_plot = plot_single_run_details(sys, sol, N)


attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit


In [30]:
display(detailed_plot)
savefig(detailed_plot, "simulation_N20.png")

BoundsError: BoundsError: attempt to access 20-element Vector{Float64} at index [1:742]